<a href="https://colab.research.google.com/github/caltunay/dd_competition_eq_damage_prediction/blob/master/04_09_eq_retake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np

In [0]:
test_vals = '/content/drive/My Drive/Colab Notebooks/Data/Competition - DrivenData - Earthquake Damage Prediction/test_values.csv'

train_vals = '/content/drive/My Drive/Colab Notebooks/Data/Competition - DrivenData - Earthquake Damage Prediction/train_values.csv'
train_labs = '/content/drive/My Drive/Colab Notebooks/Data/Competition - DrivenData - Earthquake Damage Prediction/train_labels.csv'

In [0]:
all = pd.merge(left = pd.read_csv(train_vals), right = pd.read_csv(train_labs), on = 'building_id', how = 'inner')
all.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,2
2,94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3
3,590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,0,1,0,0,0,0,1,1,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,2
4,201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,1,0,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3


In [0]:
categorik = all.select_dtypes('object').columns.tolist()

all_dummied = pd.get_dummies(data = all, prefix_sep = '|', columns = categorik, drop_first = True)

In [0]:
to_drop

all = all.drop(labels = to_drop, axis = 1)

['has_secondary_use_agriculture',
 'has_secondary_use_hotel',
 'has_secondary_use_rental',
 'has_secondary_use_institution',
 'has_secondary_use_school',
 'has_secondary_use_industry',
 'has_secondary_use_health_post',
 'has_secondary_use_gov_office',
 'has_secondary_use_use_police',
 'has_secondary_use_other']

In [0]:
# dicts for target encoding
geo1_dict = all_dummied.groupby('geo_level_1_id').mean()['damage_grade'].to_dict()
geo2_dict = all_dummied.groupby('geo_level_2_id').mean()['damage_grade'].to_dict()
geo3_dict = all_dummied.groupby('geo_level_3_id').mean()['damage_grade'].to_dict()

In [0]:
all_dummied['geo_level_1_id'] = all_dummied['geo_level_1_id'].replace(geo1_dict)
all_dummied['geo_level_2_id'] = all_dummied['geo_level_2_id'].replace(geo2_dict)
all_dummied['geo_level_3_id'] = all_dummied['geo_level_3_id'].replace(geo3_dict)

In [0]:
numerik = all_dummied.drop(labels = 'building_id geo_level_1_id geo_level_2_id geo_level_3_id damage_grade'.split(), axis = 1).nunique().sort_values(ascending = False).index[:5].tolist()

In [0]:
all_dummied[numerik].describe().T

,count,mean,std,min,25%,50%,75%,max
area_percentage,260601.0,8.018051,4.392231,1.0,5.0,7.0,9.0,100.0
age,260601.0,26.535029,73.565937,0.0,10.0,15.0,30.0,995.0
height_percentage,260601.0,5.434365,1.918418,2.0,4.0,5.0,6.0,32.0
count_families,260601.0,0.983949,0.418389,0.0,1.0,1.0,1.0,9.0
count_floors_pre_eq,260601.0,2.129723,0.727665,1.0,2.0,2.0,2.0,9.0


In [0]:
age995 = all_dummied[all_dummied['age'] == 995]
remainder = all_dummied[all_dummied['age'] != 995]

In [0]:
age_dict = remainder['age'].value_counts(normalize = True).to_dict()

In [0]:
el = []

for k, v in age_dict.items():
    
    repetition = round(v * 1390)
    temp_l = [k] * repetition
    
    el = el + temp_l

# shuffle(el)

In [0]:
from random import shuffle

In [0]:
# shuffle(el)
len(el)

1392

In [0]:
el.pop()
el.pop()

200

In [0]:
age995['age'] = el

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
all_clean = pd.concat([remainder, age995], axis = 0).reset_index(drop = True)

In [0]:
all_clean[numerik].describe().T

,count,mean,std,min,25%,50%,75%,max
area_percentage,260601.0,8.018051,4.392231,1.0,5.0,7.0,9.0,100.0
age,260601.0,21.340766,19.603808,0.0,10.0,15.0,30.0,200.0
height_percentage,260601.0,5.434365,1.918418,2.0,4.0,5.0,6.0,32.0
count_families,260601.0,0.983949,0.418389,0.0,1.0,1.0,1.0,9.0
count_floors_pre_eq,260601.0,2.129723,0.727665,1.0,2.0,2.0,2.0,9.0


In [0]:
age_series = pd.cut(x = all_clean['age'], bins = [0,10,30,80,200], include_lowest = True, labels = 'new young old ancient'.split()).rename('age_bins', inplace = True)

age_series

In [0]:
all_clean = pd.concat([all_clean, age_series], axis = 1).drop(labels = 'age', axis = 1)

In [0]:
numerik.pop(numerik.index('age'))

'age'

In [0]:
all_clean[numerik].describe().T

,count,mean,std,min,25%,50%,75%,max
area_percentage,260601.0,8.018051,4.392231,1.0,5.0,7.0,9.0,100.0
height_percentage,260601.0,5.434365,1.918418,2.0,4.0,5.0,6.0,32.0
count_families,260601.0,0.983949,0.418389,0.0,1.0,1.0,1.0,9.0
count_floors_pre_eq,260601.0,2.129723,0.727665,1.0,2.0,2.0,2.0,9.0


In [0]:
series_area = pd.cut(x = all_clean['area_percentage'], bins = [1,5,9,100], labels = 'small average big'.split(), include_lowest = True).rename('area_bins')

all_clean = pd.concat([all_clean, series_area], axis = 1).drop(labels = 'area_percentage', axis = 1)

In [0]:
all_clean.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,damage_grade,land_surface_condition|o,land_surface_condition|t,foundation_type|i,foundation_type|r,foundation_type|u,foundation_type|w,roof_type|q,roof_type|x,ground_floor_type|m,ground_floor_type|v,ground_floor_type|x,ground_floor_type|z,other_floor_type|q,other_floor_type|s,other_floor_type|x,position|o,position|s,position|t,plan_configuration|c,plan_configuration|d,plan_configuration|f,plan_configuration|m,plan_configuration|n,plan_configuration|o,plan_configuration|q,plan_configuration|s,plan_configuration|u,legal_ownership_status|r,legal_ownership_status|v,legal_ownership_status|w,age_bins,area_bins
0,802906,2.161724,2.740741,2.837838,2,5,1,1,0,0,0,0,0,0,0,0,0,1,0,3,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,young,average
1,28830,2.485273,2.487437,2.062500,2,7,0,1,0,0,0,0,0,0,0,0,0,1,0,2,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,new,average
2,94947,2.563369,2.518750,2.580882,2,5,0,1,0,0,0,0,0,0,0,0,0,1,0,3,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,new,small
3,590882,2.000960,2.107317,2.096774,2,5,0,1,0,0,0,0,1,1,0,0,0,1,0,2,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,new,average
4,201944,2.337713,2.348748,2.368852,3,9,1,0,0,0,0,0,0,0,0,0,0,1,0,3,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,young,average


In [0]:
numerik.pop(numerik.index('area_percentage'))

'area_percentage'

In [0]:
all_clean[numerik].describe().T

,count,mean,std,min,25%,50%,75%,max
height_percentage,260601.0,5.434365,1.918418,2.0,4.0,5.0,6.0,32.0
count_families,260601.0,0.983949,0.418389,0.0,1.0,1.0,1.0,9.0
count_floors_pre_eq,260601.0,2.129723,0.727665,1.0,2.0,2.0,2.0,9.0


In [0]:
all_clean['height_percentage'] = np.log1p(all_clean['height_percentage'])
all_clean['count_floors_pre_eq'] = np.log1p(all_clean['count_floors_pre_eq'])
all_clean['count_families'] = np.log1p(all_clean['count_families'])

In [0]:
all_clean = pd.get_dummies(data = all_clean, columns = ['age_bins', 'area_bins'], drop_first = True, prefix_sep = '|')

In [0]:
print(all_clean.shape)
all_clean.head()

(260601, 55)


,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,damage_grade,land_surface_condition|o,land_surface_condition|t,foundation_type|i,foundation_type|r,foundation_type|u,foundation_type|w,roof_type|q,roof_type|x,ground_floor_type|m,ground_floor_type|v,ground_floor_type|x,ground_floor_type|z,other_floor_type|q,other_floor_type|s,other_floor_type|x,position|o,position|s,position|t,plan_configuration|c,plan_configuration|d,plan_configuration|f,plan_configuration|m,plan_configuration|n,plan_configuration|o,plan_configuration|q,plan_configuration|s,plan_configuration|u,legal_ownership_status|r,legal_ownership_status|v,legal_ownership_status|w,age_bins|young,age_bins|old,age_bins|ancient,area_bins|average,area_bins|big
0,802906,2.161724,2.740741,2.837838,1.098612,1.791759,1,1,0,0,0,0,0,0,0,0,0,0.693147,0,3,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0
1,28830,2.485273,2.487437,2.062500,1.098612,2.079442,0,1,0,0,0,0,0,0,0,0,0,0.693147,0,2,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,94947,2.563369,2.518750,2.580882,1.098612,1.791759,0,1,0,0,0,0,0,0,0,0,0,0.693147,0,3,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,590882,2.000960,2.107317,2.096774,1.098612,1.791759,0,1,0,0,0,0,1,1,0,0,0,0.693147,0,2,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,201944,2.337713,2.348748,2.368852,1.386294,2.302585,1,0,0,0,0,0,0,0,0,0,0,0.693147,0,3,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0


In [0]:
# will preprocess test set as well

In [0]:
test = pd.read_csv(test_vals)
test.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,1,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,x,v,j,t,d,0,0,0,0,0,1,0,0,0,0,0,v,2,1,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,q,f,q,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


In [0]:
test = test.drop(to_drop, 1)

In [0]:
test_dummied = pd.get_dummies(data = test, columns = categorik, drop_first = True, prefix_sep = '|')

In [0]:
test_dummied.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,land_surface_condition|o,land_surface_condition|t,foundation_type|i,foundation_type|r,foundation_type|u,foundation_type|w,roof_type|q,roof_type|x,ground_floor_type|m,ground_floor_type|v,ground_floor_type|x,ground_floor_type|z,other_floor_type|q,other_floor_type|s,other_floor_type|x,position|o,position|s,position|t,plan_configuration|c,plan_configuration|d,plan_configuration|f,plan_configuration|m,plan_configuration|n,plan_configuration|o,plan_configuration|q,plan_configuration|s,plan_configuration|u,legal_ownership_status|r,legal_ownership_status|v,legal_ownership_status|w
0,300051,17,596,11307,3,20,7,6,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
1,99355,6,141,11987,2,25,13,5,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
2,890251,22,19,10044,2,5,4,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
3,745817,26,39,633,1,0,19,3,0,0,0,0,0,1,0,0,0,0,0,2,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0
4,421793,17,289,7970,3,15,8,7,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0


In [0]:
test995 = test_dummied[test_dummied['age'] == 995]
testremainder = test_dummied[test_dummied['age'] != 995]

In [0]:
el2 =[]

for k,v in age_dict.items():
    
    repetition = round(test995.shape[0] * v)
    temp_list = repetition * [k]
    
    el2 = el2 + temp_list
    
el2

In [0]:
len(el2)

459

In [0]:
el2 = el2 + [10, 10]

el2

In [0]:
shuffle(el2)

In [0]:
test995['age'] = el2

In [0]:
test_clean = pd.concat([testremainder, test995], axis = 0).reset_index(drop = True)

In [0]:
test_clean['geo_level_1_id'] = test_clean['geo_level_1_id'].replace(geo1_dict)
test_clean['geo_level_2_id'] = test_clean['geo_level_2_id'].replace(geo2_dict)
test_clean['geo_level_3_id'] = test_clean['geo_level_3_id'].replace(geo3_dict)

In [0]:
# test_clean[test_clean['geo_level_3_id'] > 3]['geo_level_3_id'] = geo3_mean

# test_clean[test_clean['geo_level_2_id'] > 3]['geo_level_2_id'] = geo2_mean

In [0]:
# test_clean['geo_level_2_id'] = 
# test_clean[['geo_level_2_id']].apply(lambda x: geo2_mean if x>3 else None).values


# df['X'] = 

test_clean['geo_level_3_id'] = pd.Series(np.where(test_clean['geo_level_3_id']>3, geo3_mean, test_clean['geo_level_3_id']))

In [0]:
test_clean['geo_level_2_id'] = pd.Series(np.where(test_clean['geo_level_2_id'] > 3, geo2_mean, test_clean['geo_level_2_id']))

# test_clean['geo_level_2_id'].describe()

In [0]:
test_clean['geo_level_2_id'].describe()

count    86868.000000
mean         2.239743
std          0.358478
min          1.000000
25%          2.019704
50%          2.212121
75%          2.481379
max          3.000000
Name: geo_level_2_id, dtype: float64

In [0]:
test_clean['geo_level_3_id'].describe()

count    86868.000000
mean         2.240055
std          0.399925
min          1.000000
25%          2.000000
50%          2.200000
75%          2.500000
max          3.000000
Name: geo_level_3_id, dtype: float64

In [0]:
geo2_mean = all_clean['geo_level_2_id'].mean()
geo3_mean = all_clean['geo_level_3_id'].mean()

In [0]:
# test_clean['geo_level_2_id'] = test_clean['geo_level_2_id'].replace({0:geo2_mean})
# test_clean[test_clean['geo_level_2_id'] == 0]

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition|o,land_surface_condition|t,foundation_type|i,foundation_type|r,foundation_type|u,foundation_type|w,roof_type|q,roof_type|x,ground_floor_type|m,ground_floor_type|v,ground_floor_type|x,ground_floor_type|z,other_floor_type|q,other_floor_type|s,other_floor_type|x,position|o,position|s,position|t,plan_configuration|c,plan_configuration|d,plan_configuration|f,plan_configuration|m,plan_configuration|n,plan_configuration|o,plan_configuration|q,plan_configuration|s,plan_configuration|u,legal_ownership_status|r,legal_ownership_status|v,legal_ownership_status|w


In [0]:
# test_clean['geo_level_3_id'] = test_clean['geo_level_3_id'].replace({0:geo3_mean})

In [0]:
age_series_test = pd.cut(x = test_clean['age'], bins = [0,10,30,80,200], include_lowest = True, labels = 'new young old ancient'.split()).rename('age_bins')

age_series_test

In [0]:
test_clean = pd.concat([test_clean, age_series_test], axis = 1)

In [0]:
series_area_test = pd.cut(x = test_clean['area_percentage'], bins = [1,5,9,100], labels = 'small average big'.split(), include_lowest = True).rename('area_bins')

test_clean = pd.concat([test_clean, series_area_test], axis = 1).drop(labels = 'area_percentage', axis = 1)

In [0]:
test_clean['height_percentage'] = np.log1p(test_clean['height_percentage'])
test_clean['count_floors_pre_eq'] = np.log1p(test_clean['count_floors_pre_eq'])
test_clean['count_families'] = np.log1p(test_clean['count_families'])

In [0]:
test_clean = pd.get_dummies(data = test_clean, columns = ['age_bins', 'area_bins'], prefix_sep = '|', drop_first = True)

In [0]:
test_clean = test_clean.drop(labels = 'age', axis = 1)

In [0]:
test_clean.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,land_surface_condition|o,land_surface_condition|t,foundation_type|i,foundation_type|r,foundation_type|u,foundation_type|w,roof_type|q,roof_type|x,ground_floor_type|m,ground_floor_type|v,ground_floor_type|x,ground_floor_type|z,other_floor_type|q,other_floor_type|s,other_floor_type|x,position|o,position|s,position|t,plan_configuration|c,plan_configuration|d,plan_configuration|f,plan_configuration|m,plan_configuration|n,plan_configuration|o,plan_configuration|q,plan_configuration|s,plan_configuration|u,legal_ownership_status|r,legal_ownership_status|v,legal_ownership_status|w,age_bins|young,age_bins|old,age_bins|ancient,area_bins|average,area_bins|big
0,300051,2.794480,2.705036,2.631579,1.386294,1.945910,0,1,0,0,0,0,0,0,0,0,0,0.693147,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0
1,99355,2.161724,2.180851,2.000000,1.098612,1.791759,0,1,0,0,0,0,0,0,0,0,0,0.693147,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1
2,890251,2.000960,2.175000,3.000000,1.098612,1.791759,0,1,0,0,0,0,0,0,0,0,0,0.693147,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,745817,1.730887,1.532194,1.437788,0.693147,1.386294,0,0,0,0,0,1,0,0,0,0,0,1.098612,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,421793,2.794480,2.689119,2.823529,1.386294,2.079442,0,1,0,0,0,0,0,0,0,0,0,0.693147,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0


In [0]:
s = 0

for tr, te in zip(all_clean.drop(labels ='damage_grade', axis = 1), test_clean.columns.tolist()):
    if tr == te:
        s = 0
    else:
        s += 1
s

0

In [0]:
from sklearn.preprocessing import StandardScaler

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

from sklearn.metrics import f1_score, confusion_matrix

from sklearn.model_selection import train_test_split as tts

Using TensorFlow backend.


In [0]:
x = all_clean.drop(labels = ['building_id', 'damage_grade'], axis = 1)
y = all_clean['damage_grade']
y_keras = pd.get_dummies(all_clean[['damage_grade']], columns = ['damage_grade'], prefix_sep = '|')

In [0]:
x_train, x_test, y_train, y_test = tts(x, y_keras, test_size = .275, random_state = 42)


In [0]:
x_train2, x_test2, y_train2, y_test2 = tts(x, y, test_size = .275, random_state = 42)

In [0]:
## Upsampled tryout

from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
sm = SMOTE(random_state = 42)
x_resampled, y_resampled = sm.fit_resample(x,y)

In [0]:
print(x_resampled.shape, y_resampled.shape)
# print(y_resampled.value_counts())

(444777, 53) (444777,)


In [0]:
y_resampled_keras = pd.get_dummies(pd.DataFrame(y_resampled), columns = [0])

x_resampled_train, x_resampled_test, y_resampled_train, y_resampled_test = tts(x_resampled, y_resampled_keras, random_state = 42, test_size = .275)

In [0]:
# y_resampled_test.rename({'0_1':1, '0_2':2, '0_3':3}, axis = 1, inplace =True)

y_resampled_test['damage_grade'] = y_resampled_test.idxmax(axis = 1)

y_resampled_test['damage_grade']

In [0]:
model = Sequential()

model.add(Dense(output_dim = 90, input_dim = 53, activation = 'relu', init = 'uniform'))
model.add(Dense(output_dim = 90, activation = 'relu'))
model.add(Dense(output_dim = 90, activation = 'relu'))
# model.add(Dense(output_dim = 60, activation = 'relu'))
# model.add(Dense(output_dim = 60, activation = 'relu'))
# model.add(Dense(output_dim = 60, activation = 'relu'))
# model.add(Dense(output_dim = 90, activation = 'relu'))
model.add(Dense(output_dim = 3, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit(x_resampled_train, y_resampled_train, batch_size = 13030, epochs = 150)


print('f1 Micro Score is : ' , f1_score(y_resampled_test['damage_grade'], pd.DataFrame(model.predict(x_resampled_test), columns = [1,2,3]).idxmax(axis = 1), average = 'micro'))
print('\n')
print('Confusion Matrix is :  ', '\n' , confusion_matrix(y_resampled_test['damage_grade'], pd.DataFrame(model.predict(x_resampled_test), columns = [1,2,3]).idxmax(axis = 1)))

In [0]:
from sklearn.model_selection import StratifiedKFold

In [0]:
for train, test in kfold.split(x_resampled, y_resampled): 
    print('xresampledtrain', x_resampled[train].shape,'yresampledtrain', y_resampled[train].shape)
    print('xresampledtest', x_resampled[test].shape, 'xresampledtest', y_resampled[test].shape)
    print('\n')

xresampledtrain (222387, 53) yresampledtrain (222387,)
xresampledtest (222390, 53) xresampledtest (222390,)


xresampledtrain (222390, 53) yresampledtrain (222390,)
xresampledtest (222387, 53) xresampledtest (222387,)




In [0]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)


# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
scores = []

for train, test in kfold.split(x_resampled, y_resampled): 

    # convert y  to one hot encoded
    temp_y = pd.get_dummies(pd.DataFrame(y_resampled),columns = [0]).values
    
    
    # create model
    clf = Sequential()
    
    clf.add(Dense(output_dim = 60, input_dim = 53, activation = 'relu', init = 'uniform'))
    clf.add(Dense(output_dim = 60, activation = 'relu'))
    clf.add(Dense(output_dim = 60, activation = 'relu'))
    clf.add(Dense(output_dim = 3, activation = 'softmax'))
    
    # Compile model
    clf.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')

    # Fit the model
    clf.fit(x_resampled[train], temp_y[train], epochs = 150, batch_size = 13030, verbose = 1)

    # make prediction, get idxmax, evaluate against test set
    y_pred = pd.DataFrame(clf.predict(x_resampled[test]), columns = [1,2,3]).idxmax(axis = 1)
    temp_score = round(f1_score(y_resampled[test], y_pred, average = 'micro'), 2)
    scores.append(temp_score)    

In [0]:
model = Sequential()

model.add(Dense(output_dim = 32, input_dim = 53, activation = 'relu', init = 'uniform'))
model.add(Dense(output_dim = 32, activation = 'relu'))
model.add(Dense(output_dim = 3, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit(x_train, y_train, batch_size = 13030, epochs = 150)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=53, activation="relu", units=32, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=32)`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=3)`
  """


Epoch 1/150
188935/188935 [==============================] - 4s 19us/step - loss: 0.9980 - acc: 0.4960
Epoch 2/150
188935/188935 [==============================] - 0s 2us/step - loss: 0.8831 - acc: 0.5683
Epoch 3/150
188935/188935 [==============================] - 0s 2us/step - loss: 0.8508 - acc: 0.5684
Epoch 4/150
188935/188935 [==============================] - 0s 2us/step - loss: 0.8177 - acc: 0.5777
Epoch 5/150
188935/188935 [==============================] - 0s 2us/step - loss: 0.7812 - acc: 0.6155
Epoch 6/150
188935/188935 [==============================] - 0s 2us/step - loss: 0.7468 - acc: 0.6500
Epoch 7/150
188935/188935 [==============================] - 0s 2us/step - loss: 0.7134 - acc: 0.6774
Epoch 8/150
188935/188935 [==============================] - 0s 2us/step - loss: 0.6816 - acc: 0.6989
Epoch 9/150
188935/188935 [==============================] - 0s 2us/step - loss: 0.6544 - acc: 0.7131
Epoch 10/150
188935/188935 [==============================] - 0s 2us/step - loss:

In [0]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)


# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
scores2 = []
x_t = x.values
y_t = y.values

for train, test in kfold.split(x_t, y_t): 

    # convert y  to one hot encoded
    temp_y = pd.get_dummies(pd.DataFrame(y_t),columns = [0]).values
    
    
    # create model
    clf = Sequential()
    
    clf.add(Dense(output_dim = 60, input_dim = 53, activation = 'relu', init = 'uniform'))
    clf.add(Dense(output_dim = 60, activation = 'relu'))
    clf.add(Dense(output_dim = 60, activation = 'relu'))
    clf.add(Dense(output_dim = 3, activation = 'softmax'))
    
    # Compile model
    clf.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')

    # Fit the model
    clf.fit(x_t[train], temp_y[train], epochs = 150, batch_size = 13030, verbose = 1)

    # make prediction, get idxmax, evaluate against test set
    y_pred = pd.DataFrame(clf.predict(x_t[test]), columns = [1,2,3]).idxmax(axis = 1)
    temp_score = round(f1_score(y_t[test], y_pred, average = 'micro'), 2)
    scores2.append(temp_score)  
    
print(scores2)
print(np.mean(scores2))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=53, activation="relu", units=60, kernel_initializer="uniform")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=60)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=60)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=3)`


Epoch 1/150
234540/234540 [==============================] - 4s 16us/step - loss: 0.9731 - acc: 0.5397
Epoch 2/150
234540/234540 [==============================] - 0s 1us/step - loss: 0.8315 - acc: 0.5819
Epoch 3/150
234540/234540 [==============================] - 0s 1us/step - loss: 0.7393 - acc: 0.6576
Epoch 4/150
234540/234540 [==============================] - 0s 1us/step - loss: 0.6611 - acc: 0.7075
Epoch 5/150
234540/234540 [==============================] - 0s 1us/step - loss: 0.6187 - acc: 0.7274
Epoch 6/150
234540/234540 [==============================] - 0s 1us/step - loss: 0.6017 - acc: 0.7337
Epoch 7/150
234540/234540 [==============================] - 0s 1us/step - loss: 0.5940 - acc: 0.7380
Epoch 8/150
234540/234540 [==============================] - 0s 1us/step - loss: 0.5878 - acc: 0.7403
Epoch 9/150
234540/234540 [==============================] - 0s 1us/step - loss: 0.5836 - acc: 0.7420
Epoch 10/150
234540/234540 [==============================] - 0s 1us/step - loss:

In [0]:
clf.predict(x)

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [0]:
np.mean(scores)

0.7629999999999999

In [0]:
clf.predict()

In [0]:
submis_x = test_clean.drop(labels = 'building_id', axis = 1).values

submis_x_df = pd.concat([test_clean['building_id'], pd.DataFrame(clf.predict(submis_x), columns = [1,2,3]).idxmax(axis = 1)], axis = 1).rename({0:'damage_grade'}, axis = 1)

submis_x_df.tail()

,building_id,damage_grade
86863,605668,2
86864,703201,2
86865,783720,2
86866,880161,1
86867,151020,2


In [0]:
submis_x_df = submis_x_df.set_index('building_id')

In [0]:
submis_x_df.to_csv('upsampled2.csv')

In [0]:
pd.merge(left = test_clean[['building_id']], right = submis_x_df, on = 'building_id', how = 'left')

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,3
3,745817,1
4,421793,3
5,871976,2
6,691228,1
7,896100,3
8,343471,2
9,766647,2


In [0]:
from sklearn.model_selection import StratifiedKFold

In [0]:
print('Confusion Matrix is :  ', '\n' , confusion_matrix(y_test['damage_grade'], pd.DataFrame(model.predict(x_test), columns = [1,2,3]).idxmax(axis = 1)))

Confusion Matrix is :   
 [[ 3700  3135    70]
 [ 1766 34947  4180]
 [  159  8701 15008]]


**Resampled version**

In [0]:
y_resampled_keras = pd.get_dummies(pd.DataFrame(y_resampled), columns = [0])

In [0]:
x_resampled_train, x_resampled_test, y_resampled_train, y_resampled_test = tts(x_resampled, y_resampled_keras, random_state = 42, test_size = .275)

In [0]:
# y_resampled_test.rename({'0_0':1, '0_1':2, '0_2':3}, axis = 1, inplace = True)
# y_resampled_test['damage_grade'] = y_resampled_test.idxmax(axis = 1)

y_resampled_test['damage_grade']

In [0]:
x_resampled_train.shape

(322463, 63)

In [0]:
model = Sequential()

model.add(Dense(output_dim = 60, input_dim = 63, activation = 'relu', init = 'uniform'))
# model.add(Dense(output_dim = 32, activation = 'relu'))
# model.add(Dense(output_dim = 32, activation = 'relu'))
# model.add(Dense(output_dim = 32, activation = 'relu'))
model.add(Dense(output_dim = 60, activation = 'relu'))
model.add(Dense(output_dim = 60, activation = 'relu'))
model.add(Dense(output_dim = 60, activation = 'relu'))
model.add(Dense(output_dim = 3, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

model.fit(x_resampled_train, y_resampled_train, batch_size = 13030, epochs = 150)

print('f1 Micro Score is : ' , f1_score(y_resampled_test['damage_grade'], pd.DataFrame(model.predict(x_resampled_test), columns = [1,2,3]).idxmax(axis = 1), average = 'micro'))
print('\n')
print('Confusion Matrix is :  ', '\n' , confusion_matrix(y_resampled_test['damage_grade'], pd.DataFrame(model.predict(x_resampled_test), columns = [1,2,3]).idxmax(axis = 1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=63, activation="relu", units=60, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=60)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=60)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=60)`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=3)`
  # Remove the CWD from sys.pat

Epoch 1/150
322463/322463 [==============================] - 1s 4us/step - loss: 0.5775 - acc: 0.6941
Epoch 2/150
322463/322463 [==============================] - 0s 2us/step - loss: 0.4164 - acc: 0.8030
Epoch 3/150
322463/322463 [==============================] - 0s 1us/step - loss: 0.3768 - acc: 0.8277
Epoch 4/150
322463/322463 [==============================] - 0s 2us/step - loss: 0.3675 - acc: 0.8328
Epoch 5/150
322463/322463 [==============================] - 0s 1us/step - loss: 0.3621 - acc: 0.8354
Epoch 6/150
322463/322463 [==============================] - 0s 2us/step - loss: 0.3593 - acc: 0.8367
Epoch 7/150
322463/322463 [==============================] - 0s 2us/step - loss: 0.3575 - acc: 0.8375
Epoch 8/150
322463/322463 [==============================] - 0s 1us/step - loss: 0.3562 - acc: 0.8381
Epoch 9/150
322463/322463 [==============================] - 0s 1us/step - loss: 0.3554 - acc: 0.8384
Epoch 10/150
322463/322463 [==============================] - 0s 1us/step - loss: 

In [0]:
submis_x = test_clean.drop(labels = 'building_id', axis = 1).values

submis_x_df = pd.concat([test_clean['building_id'], pd.DataFrame(model.predict(submis_x), columns = [1,2,3]).idxmax(axis = 1)], axis = 1).rename({0:'damage_grade'}, axis = 1)

submis_x_df.tail()

,building_id,damage_grade
86863,605668,2
86864,703201,2
86865,783720,2
86866,880161,1
86867,151020,2


In [0]:
submis_test = pd.merge(left = test_clean['building_id'], right = submis_x_df, on = 'building_id', how = 'inner')
submis_test_df = submis_test.set_index('building_id')

submis_test_df.tail()#.to_csv('submis_2.csv')

,damage_grade
building_id,
605668,2
703201,2
783720,2
880161,1
151020,2


In [0]:
model = Sequential()

model.add(Dense(output_dim = 60, input_dim = 63, activation = 'relu', init = 'uniform'))
# model.add(Dense(output_dim = 32, activation = 'relu'))
# model.add(Dense(output_dim = 32, activation = 'relu'))
# model.add(Dense(output_dim = 32, activation = 'relu'))
model.add(Dense(output_dim = 60, activation = 'relu'))
model.add(Dense(output_dim = 60, activation = 'relu'))
model.add(Dense(output_dim = 60, activation = 'relu'))
model.add(Dense(output_dim = 1, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

model.fit(x, y, batch_size = 13030, epochs = 15)

# print('f1 Micro Score is : ' , f1_score(y_resampled_test['damage_grade'], pd.DataFrame(model.predict(x_resampled_test), columns = [1,2,3]).idxmax(axis = 1), average = 'micro'))
# print('\n')
# print('Confusion Matrix is :  ', '\n' , confusion_matrix(y_resampled_test['damage_grade'], pd.DataFrame(model.predict(x_resampled_test), columns = [1,2,3]).idxmax(axis = 1)))

In [0]:
y

In [0]:
# pd.dataF

# confusion_matrix(y_test, pd.DataFrame(model.predict(x_test), columns = [1,2,3]).idxmax(axis = 1))

# y_test['damage_grade'] = 
# y_test.rename({'damage_grade|0':1, 'damage_grade|1':2, 'damage_grade|2':3}, axis = 1, inplace = True)#.idxmax(axis = 1)
y_test['damage_grade'] = y_test.idxmax(axis =1)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
# pd.DataFrame(model.predict(x_test), columns = [1,2,3]).idxmax(axis = 1)

print((confusion_matrix(y_test['damage_grade'], pd.DataFrame(model.predict(x_test), columns = [1,2,3]).idxmax(axis = 1))))

[[ 3535  3293    77]
 [ 1496 34761  4636]
 [  116  8256 15496]]


In [0]:
f1_score(y_test['damage_grade'], pd.DataFrame(model.predict(x_test), columns = [1,2,3]).idxmax(axis = 1), average = 'micro')

0.7505930287723608

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71666 entries, 111801 to 109401
Data columns (total 4 columns):
damage_grade|0    71666 non-null uint8
damage_grade|1    71666 non-null uint8
damage_grade|2    71666 non-null uint8
damage_grade      71666 non-null object
dtypes: object(1), uint8(3)
memory usage: 1.3+ MB


In [0]:
y_test.drop(labels = 'damage_grade', axis = 1, inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [0]:
submis_x = test_clean.drop(labels = 'building_id', axis = 1)

submis_df = pd.concat([test_clean['building_id'], pd.DataFrame(pd.DataFrame(model.predict(submis_x), columns = [1,2,3]).idxmax(axis = 1), columns = ['damage_grade'])], axis = 1).set_index('building_id')

# submis_df.to_csv('submis1.csv')

In [0]:
submis_df2 = pd.merge(left = pd.read_csv(test_vals)[['building_id']], right = submis_df, how = 'left', on = 'building_id').set_index('building_id')
submis_df2.to_csv('submis4.csv')